In [11]:
import numpy as np
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print("Updated sys.path:", sys.path)



from lib.envs.gridworld import GridworldEnv

Updated sys.path: ['/Users/macbook/Downloads/reinforcement-learning/DP', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '/Users/macbook/Downloads/reinforcement-learning']


In [12]:
env = GridworldEnv()

In [ ]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.

    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment.
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.

    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    states = env.nS
    iter = 0
    while True:
        change = 0
        iter += 1
        for s in range(states):
            newVal = 0
            for a in range(env.nA):
                #grab the policy, each row corresponds to a state, each column to an action
                pi = policy[s][a]
                #this is a list that contain one 4-tuple
                trans_list = env.P[s][a]

                #destructuring the 4-tuple
                transProb, sNext, reward, done = trans_list[0]

                #update the value function
                newVal += pi * (reward + discount_factor * transProb * V[sNext])

            #take max becuse it represent worse case error. so if worst case is less than theta, then all cases are less than theta
            change = max(change, np.abs(newVal - V[s]))
            V[s] = newVal

        if change < theta:
            print(iter)
            break
    return np.array(V)

In [14]:
random_policy = np.ones([env.nS, env.nA]) / env.nA

v = policy_eval(random_policy, env)

print(v)

141
[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]


In [15]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)